**Set environment**

In [8]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_CombEffect_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_CombEffect_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_CombEffect_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_CombEffect_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_CombEffect_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_CombEffect_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_CombEffect_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_CombEffect_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_CombEffect_ENCODE_FCC/references 



In [2]:
txt_fdiry = file.path(FD_RES, "assay_fcc")
dir(txt_fdiry)

[1] "MPRA_Tiling_K562_Tewhey_hannah" "STARR_ATAC_K562_Reddy_KS91"    
[3] "STARR_WHG_K562_Reddy_A001"

In [3]:
VEC_TXT_ASSAY = c(
    "STARR_ATAC_K562_Reddy_KS91",
    "STARR_WHG_K562_Reddy_A001"
)

In [13]:
for (txt_assay in VEC_TXT_ASSAY) {
    cat(txt_assay, "\n")
    txt_fdiry = file.path(FD_RES, "assay_fcc", txt_assay, "coverage", "*", "*", "summary")
    txt_fname = "data_list_count_column.rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)
    
    vec_txt_fpath = Sys.glob(txt_fglob)
    
    for (txt_fpath in vec_txt_fpath){
        txt_fdiry = dirname(txt_fpath)
        print(txt_fdiry)
    }
}

STARR_ATAC_K562_Reddy_KS91 
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/fcc_astarr_macs/astarr_macs_input_union/summary"
STARR_WHG_K562_Reddy_A001 
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary"


In [30]:
dat_col$Group

[1] "Input"  "Input"  "Input"  "Input"  "Output" "Output" "Output"

In [39]:
for (txt_assay in VEC_TXT_ASSAY) {
    
    txt_fdiry = file.path(FD_RES, "assay_fcc", txt_assay, "coverage", "*", "dnase_ENCFF274YGF", "summary")
    txt_fname = "data_list_count_column.rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)
    
    vec_txt_fpath = Sys.glob(txt_fglob)
    
    for (txt_fpath in vec_txt_fpath){

        ### read object
        lst_dat = readRDS(txt_fpath)
        dat_cnt = lst_dat$data_cnt
        dat_col = lst_dat$data_col

        ### arrange count matrix
        mat_cnt = dat_cnt %>%
            dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
            column_to_rownames(var = "Region")

        ### arrange column data
        dat_col = dat_col  %>% 
            dplyr::select(Sample, Group) %>% 
            dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
            column_to_rownames(var = "Sample")

        ### show progress: init
        cat("\n====================\n")
        cat("Assay name:    ", txt_assay, "\n")
        cat("Imported file: ", "\n")
        cat(txt_fpath,   "\n")
        flush.console()
        
        fun_display_table(head(mat_cnt))
        fun_display_table(dat_col)
        cat("\n")
        flush.console()

        ### show progress: check and break if check failed
        cat("Check sample name and dimension:", "\n")
        vec1 = colnames(mat_cnt)
        vec2 = rownames(dat_col)
        res  = all(vec1 == vec2)
        
        if (res){
            cat("Passed!", "\n")
            cat("Run DESeq2...", "\n")
            flush.console()
        } else{
            cat("Failed!", "\n")
            cat("Stop the loop.", "\n")
            flush.console()
            break()
        }
        
        ### create DESeq2 object
        dds = DESeqDataSetFromMatrix(
            countData = mat_cnt, 
            colData   = dat_col, 
            design    = ~Group)
        
        ### ensure Input group is set as reference
        dds$Group <- relevel(dds$Group, ref = "Input")

        ### run deseq analysis
        dds = DESeq(dds)

        ### save deseq object
        txt_fdiry = dirname(txt_fpath)
        txt_fname = "data_deseq2.rds"
        txt_fpath = file.path(txt_fdiry, txt_fname)

        #obj = dds
        #saveRDS(obj, txt_fpath)
    }
    
}


Assay name:     STARR_ATAC_K562_Reddy_KS91 
Imported file:  
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary/data_list_count_column.rds 


,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
chr1:181400-181530,166,229,227,218,220,180,7,11,16,8
chr1:778660-778800,3014,3520,3601,3482,3447,3189,1937,2413,2644,4819
chr1:779137-779200,1137,1390,1465,1383,1383,1268,84,128,182,343
chr1:827460-827554,2402,2947,3119,2971,2918,2721,636,717,820,1657
chr1:842880-843060,448,526,566,547,510,471,99,190,217,284
chr1:869800-869980,880,1034,1068,1034,994,935,46,38,51,105


,Group
Input.rep1,Input
Input.rep2,Input
Input.rep3,Input
Input.rep4,Input
Input.rep5,Input
Input.rep6,Input
Output.rep1,Output
Output.rep2,Output
Output.rep3,Output
Output.rep4,Output



Check sample name and dimension: 
Passed! 
Run DESeq2... 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




Assay name:     STARR_WHG_K562_Reddy_A001 
Imported file:  
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary/data_list_count_column.rds 


,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Output.rep1,Output.rep2,Output.rep3
chr1:181400-181530,3,18,12,15,34,29,51
chr1:778660-778800,10,35,27,30,195,181,312
chr1:779137-779200,8,31,17,23,56,77,114
chr1:827460-827554,10,25,24,25,99,89,128
chr1:842880-843060,7,21,19,14,48,40,96
chr1:869800-869980,9,25,46,33,43,41,79


,Group
Input.rep1,Input
Input.rep2,Input
Input.rep3,Input
Input.rep4,Input
Output.rep1,Output
Output.rep2,Output
Output.rep3,Output



Check sample name and dimension: 
Passed! 
Run DESeq2... 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [22]:
vec1 = colnames(mat_cnt)
vec2 = dat_col$conditions
print(all(vec1 == vec2))

[1] TRUE
